---
# ライブラリと前処理済み画像データの読み込み
---
~~~python
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve, roc_auc_score

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch import from_numpy, manual_seed
from torch.utils.data import DataLoader, TensorDataset

# 前処理済み画像データの読み込み
X_train, X_test, y_train, y_test = np.load("./animal.npy")

# 形状を（件数、縦、横、RGB）から（件数、RGB、縦、横）に変換する
X_train = np.transpose(X_train, (0, 3, 1, 2))
X_test = np.transpose(X_test, (0, 3, 1, 2))

#numpyのデータをテンソル型に変換
X_train = from_numpy(X_train.astype(np.float32))
y_train = from_numpy(y_train.astype(np.int64))
X_test = from_numpy(X_test.astype(np.float32))
y_test = from_numpy(y_test.astype(np.int64))
~~~

In [2]:

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve, roc_auc_score

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch import from_numpy, manual_seed
from torch.utils.data import DataLoader, TensorDataset

# 前処理済み画像データの読み込み
X_train, X_test, y_train, y_test = np.load("./animal.npy", allow_pickle=True)

# 形状を（件数、縦、横、RGB）から（件数、RGB、縦、横）に変換する
X_train = np.transpose(X_train, (0, 3, 1, 2))
X_test = np.transpose(X_test, (0, 3, 1, 2))

#numpyのデータをテンソル型に変換
X_train = from_numpy(X_train.astype(np.float32))
y_train = from_numpy(y_train.astype(np.int64))
X_test = from_numpy(X_test.astype(np.float32))
y_test = from_numpy(y_test.astype(np.int64))

---
# 学習モデルを定義
---
~~~python
# CNNを実装する
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 9 * 9, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 3)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        x = x.view(-1, 16 * 9 * 9)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
~~~

In [3]:
# CNNを実装する
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 9 * 9, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 3)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        x = x.view(-1, 16 * 9 * 9)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

---
# 学習済みモデルを読み込み
---
~~~python
model = Net()
PATH = './animal_cnn.pth'
model.load_state_dict(torch.load(PATH))
~~~

In [4]:
model = Net()
PATH = './animal_cnn.pth'
model.load_state_dict(torch.load(PATH))

<All keys matched successfully>

---
# 精度確認
---
~~~python
#テストデータで予測結果を作成
y_prob , y_pred = torch.max(model(X_test).data, 1)

accuracy = accuracy_score(y_test, y_pred)

print("正確度：",accuracy)

# 正確度（正解率）を算出　(TP + TN) / 全体
accuracy = accuracy_score(y_test, y_pred)

# 適合度を算出　TP / (TP + FP)
precision = precision_score(y_test, y_pred, average=None)

#再現度を算出　TP / (TP + FN)
recall = recall_score(y_test, y_pred, average=None)

#F1スコアを算出 (2 - 適合度 × 再現度) / (適合度 × 再現度)
f1 = f1_score(y_test, y_pred, average=None)

print("正確度（Accuracy）                   適合度（Precision） ") 
print(accuracy, "                 ", precision) 
print("再現度（Recall）                     F1スコア（F1） ") 
print(recall, " ", f1) 
~~~

In [5]:
#テストデータで予測結果を作成
y_prob , y_pred = torch.max(model(X_test).data, 1)

accuracy = accuracy_score(y_test, y_pred)

print("正確度：",accuracy)

# 正確度（正解率）を算出　(TP + TN) / 全体
accuracy = accuracy_score(y_test, y_pred)

# 適合度を算出　TP / (TP + FP)
precision = precision_score(y_test, y_pred, average=None)

#再現度を算出　TP / (TP + FN)
recall = recall_score(y_test, y_pred, average=None)

#F1スコアを算出 (2 - 適合度 × 再現度) / (適合度 × 再現度)
f1 = f1_score(y_test, y_pred, average=None)

print("正確度（Accuracy）                   適合度（Precision） ") 
print(accuracy, "                 ", precision) 
print("再現度（Recall）                     F1スコア（F1） ") 
print(recall, " ", f1) 

正確度： 0.5944444444444444
正確度（Accuracy）                   適合度（Precision） 
0.5944444444444444                   [0.60344828 0.47916667 0.66216216]
再現度（Recall）                     F1スコア（F1） 
[0.63636364 0.37704918 0.765625  ]   [0.61946903 0.42201835 0.71014493]
